Alex Thornton (apt2141)  
Columbia University Fall 2021  
ELEN E6893 - Big Data Analytics  
Research Project: SpotifyClassifier

1. Connect to GCP

In [1]:
from google.colab import auth
auth.authenticate_user()
!gcloud config set project spotifyclassifier
!pip install spotipy

Updated property [core/project].


In [2]:
import os
os.environ['SPOTIPY_CLIENT_ID'] = 'SECRET'
os.environ['SPOTIPY_CLIENT_SECRET'] = 'SECRET'

In [3]:
!pip install --upgrade google-api-python-client

In [12]:
from googleapiclient.discovery import build
gcs_service = build('storage', 'v1')
from googleapiclient.http import MediaFileUpload
from googleapiclient.http import MediaIoBaseDownload

3. Import Packages

In [5]:
import requests
import json
import spotipy
import numpy as np
from spotipy.oauth2 import SpotifyClientCredentials

3. Setup Spotify API

In [6]:
auth_manager = SpotifyClientCredentials()
sp = spotipy.Spotify(auth_manager=auth_manager)

4. Do Stuff

In [7]:
genres = ["techno","j-dance","deep-house",\
    "rock","rock-n-roll","rockabilly","j-rock","hard-rock","psych-rock","power-pop","guitar","drum-and-bass",\
    "country","bluegrass","acoustic","folk","honky-tonk",\
    "pop","k-pop","j-pop","cantopop","party","pop-film","r-n-b","club","hip-hop",\
    "jazz","soul","funk","blues","afrobeat",\
    "latin","latino","reggaeton","samba","salsa","bossanova",\
    "classical","opera",\
    "show-tunes","disney"]

!mkdir /track-files
for genre in genres:
    with open('/track-files/'+genre+'.json', 'wb') as f:
        request = gcs_service.objects().get_media(bucket='track-lists-full',object=genre+'.json')
        media = MediaIoBaseDownload(f, request)
        done = False
        while not done:
            _, done = media.next_chunk()

    with open('/track-files/'+genre+'.json', "r") as tracklist:
        tracklist = json.load(tracklist)
        num_songs = len(tracklist.keys())
    print("Genre: "+genre+", # Tracks: "+str(num_songs))



mkdir: cannot create directory ‘/track-files’: File exists
Genre: techno, # Tracks: 500
Genre: j-dance, # Tracks: 369
Genre: deep-house, # Tracks: 500
Genre: rock, # Tracks: 500
Genre: rock-n-roll, # Tracks: 500
Genre: rockabilly, # Tracks: 500
Genre: j-rock, # Tracks: 500
Genre: hard-rock, # Tracks: 100
Genre: psych-rock, # Tracks: 500
Genre: power-pop, # Tracks: 331
Genre: guitar, # Tracks: 500
Genre: drum-and-bass, # Tracks: 500
Genre: country, # Tracks: 500
Genre: bluegrass, # Tracks: 500
Genre: acoustic, # Tracks: 500
Genre: folk, # Tracks: 500
Genre: honky-tonk, # Tracks: 500
Genre: pop, # Tracks: 500
Genre: k-pop, # Tracks: 500
Genre: j-pop, # Tracks: 500
Genre: cantopop, # Tracks: 257
Genre: party, # Tracks: 500
Genre: pop-film, # Tracks: 96
Genre: r-n-b, # Tracks: 500
Genre: club, # Tracks: 100
Genre: hip-hop, # Tracks: 967
Genre: jazz, # Tracks: 500
Genre: soul, # Tracks: 500
Genre: funk, # Tracks: 80
Genre: blues, # Tracks: 500
Genre: afrobeat, # Tracks: 500
Genre: latin, # 

In [7]:
!ls /track-files

acoustic.json	 drum-and-bass.json  k-pop.json       rockabilly.json
afrobeat.json	 folk.json	     latin.json       rock.json
bluegrass.json	 funk.json	     latino.json      rock-n-roll.json
blues.json	 guitar.json	     opera.json       salsa.json
bossanova.json	 hard-rock.json      party.json       samba.json
cantopop.json	 hip-hop.json	     pop-film.json    show-tunes.json
classical.json	 honky-tonk.json     pop.json	      soul.json
club.json	 jazz.json	     power-pop.json   techno.json
country.json	 j-dance.json	     psych-rock.json
deep-house.json  j-pop.json	     reggaeton.json
disney.json	 j-rock.json	     r-n-b.json


In [8]:
!mkdir /audio-features

In [9]:
for genre in genres:
    !mkdir /audio-features/$genre

In [8]:
#Map subgenres to their supergenre parent node
supergenre = {
    'alternative':'alternative',
    'indie':'alternative',
    'alt-rock':'alternative',
    'punk':'alternative',
    'grunge':'alternative',
    'punk-rock':'alternative',
    'indie-pop':'alternative',
    'emo':'alternative',
    'goth':'alternative',
    'hardcore':'hardcore',
    'death-metal':'hardcore',
    'grindcore':'hardcore',
    'black-metal':'hardcore',
    'heavy-metal':'hardcore',
    'metalcore':'hardcore',
    'metal':'hardcore',
    'metal-misc':'hardcore',
    'dance':'dance',
    'electronic':'dance',
    'electro':'dance',
    'edm':'dance',
    'garage':'dance',
    'house':'dance',
    'chicago-house':'dance',
    'breakbeat':'dance',
    'dubstep':'dance',
    'techno':'dance',
    'j-dance':'dance',
    'deep-house':'dance',
    'ambient':'easy-listening',
    'chill':'easy-listening',
    'sleep':'easy-listening',
    'new-age':'easy-listening',
    'study':'easy-listening',
    'piano':'easy-listening',
    'rock':'rock',
    'rock-n-roll':'rock',
    'rockabilly':'rock',
    'j-rock':'rock',
    'hard-rock':'rock',
    'psych-rock':'rock',
    'power-pop':'rock',
    'guitar':'rock',
    'drum-and-bass':'rock',
    'country':'country',
    'bluegrass':'country',
    'acoustic':'country',
    'folk':'country',
    'honky-tonk':'country',
    'jazz':'jazz',
    'soul':'jazz',
    'funk':'jazz',
    'blues':'jazz',
    'afrobeat':'jazz',
    'latin':'latin',
    'latino':'latin',
    'reggaeton':'latin',
    'samba':'latin',
    'salsa':'latin',
    'bossanova':'latin',
    'classical':'classical',
    'opera':'classical',
    'disney':'musical',
    'show-tunes':'musical'
}

In [17]:
genres = ["hip-hop","pop","k-pop","j-pop","cantopop","party","pop-film","r-n-b","club","hip-hop"]

In [ ]:
for genre in genres:
    print("Starting " +genre)
    with open('/track-files/'+genre+'.json', "r") as tracklist:
        tracklist = json.load(tracklist)
        for track in tracklist:
            track_id = tracklist[track]
            track_data = sp.audio_features(track_id)
            track_data = track_data[0]
            try:
                analysis = sp.audio_analysis(track_id)

                # Save audio features locally
                with open("/audio-features/"+track_id+".json", 'w') as f:
                    #Add some of our own labels
                    track_data['subgenre-label'] = genre 
                    track_data['super-genre-label'] = supergenre[genre]
                    track_data['song-title'] = track

                    #Fetch some additional info for us
                    basic_info = sp.track(track_id)
                    track_data['popularity'] = basic_info['popularity']
                    track_data['explicit'] = basic_info['explicit']
                    album = basic_info['album']
                    track_data['release_date'] = album['release_date']
                    track_data['release_date_precision'] = album['release_date_precision']

                    #Add in some Spotify Audio Analysis Vectors

                    num_seg = 0
                    pitches = np.zeros(12)
                    timbre = np.zeros(12)

                    analysis_data = analysis['track']
                    time_signature = analysis_data['time_signature']
                    key = analysis_data['key']
                    mode = analysis_data['mode']

                    if "segments" in analysis:
                        for _, j in enumerate(analysis['segments']):
                            pitches += np.array(j['pitches'])
                            timbre += np.array(j['timbre'])
                            num_seg+=1

                    track_data['avg_pitches'] = list(pitches/num_seg)
                    track_data['avg_timbre'] = list(timbre/num_seg)
                    #print(track_data)

                    json.dump(track_data, f, indent=4)

                # Upload audio features to GCP bucket
                media = MediaFileUpload("/audio-features/"+track_id+".json", 
                                  mimetype='application/json',
                                  resumable=True)

                request = gcs_service.objects().insert(bucket='audio-features-fullset', 
                                                      name=genre+"/"+track_id+".json",
                                                      media_body=media)
                print("Loaded track "+track+", Genre: "+genre)
                response = None
                while response is None:
                    _, response = request.next_chunk()
            except:
                print("No analysis available, zero padding "+track+", Genre: "+genre)
                
                # Save audio features locally
                with open("/audio-features/"+track_id+".json", 'w') as f:
                    #Add some of our own labels
                    track_data['subgenre-label'] = genre 
                    track_data['super-genre-label'] = supergenre[genre]
                    track_data['song-title'] = track

                    #Fetch some additional info for us
                    basic_info = sp.track(track_id)
                    track_data['popularity'] = basic_info['popularity']
                    track_data['explicit'] = basic_info['explicit']
                    album = basic_info['album']
                    track_data['release_date'] = album['release_date']
                    track_data['release_date_precision'] = album['release_date_precision']

                    #Add in some Spotify Audio Analysis Vectors
                    analysis = sp.audio_analysis(track_id)

                    pitches = np.zeros(12)
                    timbre = np.zeros(12)

                    track_data['avg_pitches'] = list(pitches)
                    track_data['avg_timbre'] = list(timbre)
                    #print(track_data)

                    json.dump(track_data, f, indent=4)

            
            response = None
            while response is None:
                _, response = request.next_chunk()


Starting jazz
Loaded track Along Came Betty, Genre: jazz
Loaded track Directions, Genre: jazz
Loaded track 'Round Midnight, Genre: jazz
Loaded track Next Please, Genre: jazz
Loaded track Nengueleru, Genre: jazz
Loaded track Are You Real - Remastered, Genre: jazz
Loaded track Hat And Beard, Genre: jazz
Loaded track Summer Sun, Genre: jazz
Skipped track Summer Sun, Genre: jazz
Loaded track Fly Me To The Moon - 2008 Remastered, Genre: jazz
Skipped track Fly Me To The Moon - 2008 Remastered, Genre: jazz
Loaded track K, Genre: jazz
Loaded track Slow Dance, Genre: jazz
Loaded track The Girl From Ipanema - Stereo Version, Genre: jazz
Skipped track The Girl From Ipanema - Stereo Version, Genre: jazz
Loaded track The Gypsy, Genre: jazz
Skipped track The Gypsy, Genre: jazz
Loaded track Brazilian Suite, Genre: jazz
Loaded track The Toy Trumpet, Genre: jazz
Loaded track Channel 1 Suite, Genre: jazz
Loaded track Blue In Green (Take 3), Genre: jazz
Loaded track One and One, Genre: jazz
Skipped track